In [9]:
import numpy as np
from gurobipy import *

In [10]:
def readFile(nom_file):
    nom_file+=".txt"
    #file = open(nom_file,"r")
    with open(nom_file, 'r') as f:
        l = [[int(num) for num in line.split()] for line in f]
        #print(l)
    return np.asarray(l)
    f.close()

In [11]:
def pmr(matrice):
    nb_alt,nb_citere=matrice.shape
    matrice_pmr=np.zeros((nb_alt,nb_alt))
    for i in range(nb_alt):
        for j in range(nb_alt):
            if i!=j:               
                cpt=np.ndarray.max(np.array(matrice[i,:]-matrice[j,:]))
                matrice_pmr[i,j]=cpt
    return matrice_pmr


def max_regret(pmrM,l):
    nb_alt=pmrM.shape[0]
    max_regret=np.zeros(nb_alt)
    worst_ad=np.zeros(nb_alt)
    for i in range(nb_alt):
        max_regret[i]=np.ndarray.max(pmrM[i,:])
        worst_ad[i]=int(np.argmax(pmrM[i,:]))
    return max_regret,worst_ad

In [31]:
def Query(poid,sol1,sol2):
    #print(poid*sol1,poid*sol2)
    print(np.sum(poid*sol1),np.sum(poid*sol2))
    if(np.sum(poid*sol1)>np.sum(poid*sol2)):  
        return False
    else:
        return True
    
def expression(q,ligne):
    expr=LinExpr()
    for i in range(len(q)):
        #print(i,q[i],ligne[i])
        expr+=q[i]*ligne[i]
    #print(expr)
    return expr

def newPMR(m,matrice,pmr,delete,q):
    matrice=np.delete(matrice,delete,0)
    pmr=np.delete(pmr,delete,0)
    pmr=np.delete(pmr,delete,1)
    #print(pmr)
    a,b=pmr.shape
    for i in range(a):
        for j in range(b):
            if(i!=j):
                expr1=expression(q,matrice[i,:])
                expr2=expression(q,matrice[j,:])
                #print(matrice[i,:],matrice[j,:])
                m.setObjective(expr1-expr2,GRB.MAXIMIZE)
                m.optimize()
                pmr[i,j]=m.objVal
    return matrice,pmr

In [72]:
class MiniMaxRegret(object):
    def __init__(self,matrice,list_poids):
        """
        matrice:matrice de alternatives et leurs evaluations par criteres
        resultat:enregistrer les labels des alternatives
        """
        self.matrice=matrice
        self.nb_alt,self.nb_c=matrice.shape
        self.label=list(range(1,self.nb_alt+1))
        #self.matrice_pmr=pmr(matrice)
        #self.matrice_mr,self.worst_ad=max_regret(self.matrice_pmr,self.label)
        self.poids=list_poids
        #self.sol=np.argmin(self.matrice_mr)
        
    
    def MMR(self):
        m = Model("s0")
        #n=matrice.shape[1]
        n=self.nb_c
        var=[]
        for j in range(n):
            var.append(m.addVar(vtype=GRB.CONTINUOUS, name="w%d"%(j+1)))
        expr=LinExpr()
        
        #la premiere constrainte : la somme est egale a 1
        for j in range(n):
            expr+=var[j]
        m.addConstr(expr==1)
        
        i=0
        pmr_matrice=pmr(self.matrice)
        #print(pmr_matrice)
        while(pmr_matrice.shape[0]>=2):
        #while(i>=0):
            matrice_mr,worst_ad=max_regret(pmr_matrice,self.label)
            print(matrice_mr)
            sol=np.argmin(matrice_mr)
            print(self.label)
            print("iteration",i+1)
            print("the solution de min-max regret est solution",self.label[sol])
            pire_ad=int(worst_ad[sol])
            print("son pire adversaire est la solution",self.label[pire_ad])
            if(self.label[sol]==self.label[pire_ad]):
                print("best solution est solution",self.label[sol])
                break
            
            expr1=expression(var,self.matrice[sol,:])
            expr2=expression(var,self.matrice[pire_ad,:])
            #m.setObjective(expr1-expr2,GRB.MAXIMIZE)
            #m.setObjective(expr1-expr2,GRB.MAXIMIZE)
            if(Query(self.poids,self.matrice[sol,:],self.matrice[pire_ad,:])):
                print(self.matrice[sol,:],self.matrice[pire_ad,:])
                print("solution",self.label[sol],"est plus prefere que",self.label[pire_ad])
                m.addConstr(expr1<=expr2)
                self.label.pop(pire_ad)
                delete=pire_ad
                
            else:
                print("solution",self.label[pire_ad],"est plus prefere que",self.label[sol])
                self.label.pop(sol)
                m.addConstr(expr2<=expr1)
                delete=sol
           
            self.matrice,pmr_matrice=newPMR(m,self.matrice,pmr_matrice,delete,var)
            #MR
            i+=1
            print(self.matrice)
            print(pmr_matrice)
            
            

In [73]:
matrice=readFile("exempleinstance")
poids=np.array([0.326851,0.318176,0.354973])

In [74]:
m=MiniMaxRegret(matrice,poids)

In [75]:
m.MMR()

[19. 22. 20. 19. 19.]
[1, 2, 3, 4, 5]
iteration 1
the solution de min-max regret est solution 1
son pire adversaire est la solution 4
33.051922000000005 34.805371
[32 42 26] [45 23 36]
solution 1 est plus prefere que 4
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0xf2456d39
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds
Optimal objective  6.000000000e+00
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective

  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5775862e+31   3.094828e+30   1.577586e+01      0s
       2   -1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.03 seconds
Optimal objective -1.000000000e+00
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 3 rows, 3 columns and 9 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.6000000e+30   1.462500e+30   5.600000e+00      0s
       1    6.3636364e-01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds
Optimal objective  6.363636364e-01
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Opt